In [ ]:
# You can use 'pip install deep-forest datatable' in internet mode.
!pip install ../input/pypi-buf/deep_forest-0.1.0-cp37-cp37m-manylinux1_x86_64.whl ../input/pypi-buf/datatable-0.11.1-cp37-cp37m-manylinux2010_x86_64.whl

In [ ]:
import numpy as np
import pandas as pd
import datatable as dt

In [ ]:
train = dt.fread('../input/jane-street-market-prediction/train.csv').to_pandas()
train = train.query('date > 85').reset_index(drop = True) 
train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns}) #limit memory use
train.fillna(train.mean(),inplace=True)
train = train.query('weight > 0').reset_index(drop = True)

train['action_conservative'] = ((train['resp_1']>0)&(train['resp_2']>0)&(train['resp_3']>0)&(train['resp_4']>0)&(train['resp']>0)).astype(int)

features = [c for c in train.columns if 'feature' in c]
resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

train_features = train[features].values

train_resps_sparse = np.stack([(train[c]>0).astype('int') for c in resp_cols]).T 
train_resps_sparse_avg = train_resps_sparse.mean(axis=1)

train_resps_dense = train[resp_cols]
train_resps_dense_avg = train_resps_dense.mean(axis=1)

train['action_radical'] = np.where(train_resps_dense_avg >= 0.6, 1, 0).astype(int)

train_label_conservative = train['action_conservative'].values
train_label_radical = train['action_radical'].values

f_mean = np.mean(train[features[1:]].values,axis=0)

In [ ]:
from deepforest import CascadeForestClassifier

In [ ]:
model = CascadeForestClassifier(random_state=1, n_jobs=-1)
# model.fit(train_features, train_label_conservative)

In [ ]:
# import janestreet
# from tqdm import tqdm
# env = janestreet.make_env()
# for (test_df, pred_df) in tqdm(env.iter_test()):
#     if test_df['weight'].item() > 0:
#         x_tt = test_df.loc[:, features].values
#         if np.isnan(x_tt[:, 1:].sum()):
#             x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
#         pred_df.action = model.predict(x_tt)
#     else:
#         pred_df.action = 0
#     env.predict(pred_df)